# Results

Final QALY estimates with uncertainty quantification.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from whatnut import MonteCarloSimulation, DEFAULT_PARAMS

sim = MonteCarloSimulation(seed=42)
result = sim.run(DEFAULT_PARAMS)

## Summary Table

In [ ]:
summary = []
for r in result.results:
    summary.append({
        'Nut': r.nut_id.capitalize(),
        'Median QALYs': f"{r.median:.2f}",
        '95% CI': f"[{r.ci_95[0]:.1f}, {r.ci_95[1]:.1f}]",
        'P(positive)': f"{r.p_positive:.1%}",
        'P(>1 year)': f"{r.p_gt_1_year:.1%}",
    })

pd.DataFrame(summary)

## Forest Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

nuts = [r.nut_id.capitalize() for r in result.results]
medians = [r.median for r in result.results]
ci_low = [r.ci_95[0] for r in result.results]
ci_high = [r.ci_95[1] for r in result.results]

y_pos = np.arange(len(nuts))

# Plot error bars
ax.errorbar(medians, y_pos, xerr=[np.array(medians) - np.array(ci_low), 
                                   np.array(ci_high) - np.array(medians)],
            fmt='o', capsize=5, capthick=2, markersize=10, color='teal')

# Add reference line for category effect
ax.axvline(result.category_effect.median, color='gray', linestyle='--', 
           label=f'Any nut: {result.category_effect.median:.1f} QALYs')

ax.set_yticks(y_pos)
ax.set_yticklabels(nuts)
ax.set_xlabel('QALYs (95% CI)')
ax.set_title('Lifetime QALY Impact by Nut Type')
ax.legend()
ax.invert_yaxis()

plt.tight_layout()
plt.show()

## Key Insights

1. **Walnut leads** with ~2.9 QALYs, driven by strong CVD outcome data and unique omega-3 content
2. **The spread is small** - only ~0.7 QALYs between best (walnut) and worst (cashew)
3. **Any nut beats no nuts** - the category effect (~2.5 QALYs) far exceeds nut choice effects
4. **Uncertainty varies** - nuts with less evidence (pecan, macadamia) have wider CIs

## Sensitivity Analysis

In [ ]:
from whatnut.simulation import SimulationParams

# Test different confounding assumptions
# 0.6 = 60% causal effect, 40% confounding
# 0.99 = nearly all causal
confounding_levels = [0.6, 0.7, 0.8, 0.9, 0.95]
sensitivity_results = []

for conf in confounding_levels:
    params = SimulationParams(confounding_adjustment=conf, n_simulations=5000)
    res = sim.run(params)
    walnut = next(r for r in res.results if r.nut_id == 'walnut')
    sensitivity_results.append({
        'Confounding': f"{conf:.0%} causal",
        'Walnut QALYs': f"{walnut.median:.2f}",
        'Category Effect': f"{res.category_effect.median:.2f}",
    })

pd.DataFrame(sensitivity_results)